In [2]:
import os
import glob
import pandas as pd

In [53]:
old = pd.read_csv("../../src/processing_area_june_28.csv")
new = pd.read_csv("../../src/processing_area_nov_10.csv")




In [31]:

import numpy as np
len(np.unique(new['Y']))

1674

In [54]:
new.head(5)

,field_1,X,Y,Y_tile,X_tile,sovereignt,n,distance,feature_x,feature_y,nearest_x,nearest_y
0,0,-116.9722,37.4722,1624,0,United States of America,1,0.0,-116.9722,37.4722,-116.9722,37.4722
1,1,-116.9167,37.4722,1624,1,United States of America,1,0.0,-116.9167,37.4722,-116.9167,37.4722
2,2,-116.8611,37.4722,1624,2,United States of America,1,0.0,-116.8611,37.4722,-116.8611,37.4722
3,3,-116.5833,37.4722,1624,3,United States of America,1,0.0,-116.5833,37.4722,-116.5833,37.4722
4,4,-116.5278,37.4722,1624,4,United States of America,1,0.0,-116.5278,37.4722,-116.5278,37.4722


In [33]:
new['Y'] = np.around(new['Y'], 4)
old['Y'] = np.around(old['Y'], 4)
new['X'] = np.around(new['X'], 4)
old['X'] = np.around(old['X'], 4)

In [36]:
old_y = np.unique(old['Y'])
new_y = np.unique(new['Y'])

old_x = np.unique(old['X'])
new_x = np.unique(new['X'])

to_remove_y = [x for x in new_y if x not in old_y]
to_remove_x = [x for x in new_x if x not in old_x]
print(len(to_remove_x))

for i in to_remove_y:
    new = new[new.Y != i]
    
for i in to_remove_x:
    new = new[new.X != i]

155


In [37]:
new.to_csv("temp.csv")



In [38]:
new = pd.merge(new, old, on = ['X', 'Y'], how = 'left')




In [39]:
def id_tile_number(new, old, idx):
    if np.isnan(new['Y_tile'][idx]):
        new_y = new['Y'][idx]
        new_x = new['X'][idx]
        y_tile = np.unique(old[old['Y'] == new_y]['Y_tile'])
        x_tile = np.unique(old[old['X'] == new_x]['X_tile'])
        assert len(y_tile) == 1
        assert len(x_tile) == 1
        new['X_tile'][idx] = x_tile[0]
        new['Y_tile'][idx] = y_tile[0]


In [43]:
import warnings 
warnings.simplefilter(action='ignore') 
from tqdm.notebook import tnrange
for row in tnrange(len(new)):
    id_tile_number(new, old, row)

  0%|          | 0/1249302 [00:00<?, ?it/s]

In [55]:
new.head(5)

,field_1,X,Y,Y_tile,X_tile,sovereignt,n,distance,feature_x,feature_y,nearest_x,nearest_y
0,0,-116.9722,37.4722,1624,0,United States of America,1,0.0,-116.9722,37.4722,-116.9722,37.4722
1,1,-116.9167,37.4722,1624,1,United States of America,1,0.0,-116.9167,37.4722,-116.9167,37.4722
2,2,-116.8611,37.4722,1624,2,United States of America,1,0.0,-116.8611,37.4722,-116.8611,37.4722
3,3,-116.5833,37.4722,1624,3,United States of America,1,0.0,-116.5833,37.4722,-116.5833,37.4722
4,4,-116.5278,37.4722,1624,4,United States of America,1,0.0,-116.5278,37.4722,-116.5278,37.4722


,X,Y,Y_tile,X_tile,sovereignt
0,-116.9722,37.4722,1624,0,United States of America
1,-116.9167,37.4722,1624,1,United States of America
2,-116.8611,37.4722,1624,2,United States of America
3,-116.5833,37.4722,1624,3,United States of America
4,-116.5278,37.4722,1624,4,United States of America


In [57]:
new.to_csv("../../src/processing_area_nov_10.csv", index = False)




In [71]:
base_folder = "../project-monitoring/tof/"
from shutil import copyfile, copytree

for folder_y in os.listdir(base_folder):
    if "." not in folder_y:
        for folder_x in os.listdir(base_folder + folder_y):
            if "." not in folder_x:
                for file in os.listdir(base_folder + folder_y + "/" + folder_x):
                    if file[-4:] == ".tif":
                        source_file = file
                        source = old[old['X_tile'].astype(str) == folder_y]
                        source = source[source['Y_tile'].astype(str) == folder_x]
                        source = source.reset_index(drop = True)
                        source_y = source['Y'][0]
                        source_x = source['X'][0]
                        dest = new[np.isclose(new['Y'], source_y)]
                        dest = dest[np.isclose(dest['X'], source_x)]
                        dest = dest.reset_index(drop = True)
                        dest_x = dest['X_tile'][0]
                        dest_y = dest['Y_tile'][0]
                        #print(folder_y, dest_x)
                        #print(folder_x, dest_y)
                        
                        new_dir = "../project-monitoring/tiles/" + str(dest_x) + "/" + str(dest_y) + "/"
                        new_file = f'{new_dir}{dest_x}X{dest_y}Y_v1.tif'
                        old_file = f'{base_folder}{folder_y}/{folder_x}/{file}'
                        
                        
                        if not os.path.exists(os.path.realpath(new_dir)):
                            os.makedirs(os.path.realpath(new_dir))
                        copyfile(old_file, new_file)

# Create temp copy of individual country for reprocessing

In [4]:
df = new[new['country'] == 'Rwanda']
df.head(5)

,Y,X,country,Y_tile,X_tile
138843,-1.138889,30.416667,Rwanda,943,2233
138844,-1.194444,30.305556,Rwanda,942,2231
138845,-1.194444,30.361111,Rwanda,942,2232
138846,-1.194444,30.416667,Rwanda,942,2233
138847,-1.250000,30.250000,Rwanda,941,2230


In [10]:
from shutil import copyfile, copytree

for index, row in df.iterrows():
    x = str(int(row['X_tile']))
    y = str(int(row['Y_tile']))
    old_path = f'../../project-monitoring/tof-output/{str(x)}/{str(y)}/'
    new_path = f'../../project-monitoring/rwanda/{str(x)}/{str(y)}/'
    if os.path.isdir(old_path):
        copytree(old_path, new_path)
